In [11]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 23 12:08:39 2024

@author: Utente
"""

import numpy as np

AmeEur='e'
S=75
T2=5
t1=0
r=0.07
q=0
b=r-q
k=0.03
m=1
sigma=0.2
F=100
X=100
Coupon=6
N=700

CRatio=F/X

OptionValue=np.zeros(N+1)
ConversionProbability=np.zeros(N+1)

dt=T2/N


#CRR
u=np.exp(sigma*np.sqrt(dt))
d=1/u
p=(np.exp((r-q)*dt)-d)/(u-d)

#Haahtela
u = np.exp(r * dt) * (1 + np.sqrt(np.exp((sigma ** 2) * dt) - 1))
d = np.exp(r * dt) * (1 - np.sqrt(np.exp((sigma ** 2) * dt) - 1))
p = (np.exp((r - q) * dt) - d) / (u - d)

#EQP

p=0.5;
u=np.exp(((b)-sigma**2/2)*dt+sigma*np.sqrt(dt));
d=np.exp(((b)-sigma**2/2)*dt-sigma*np.sqrt(dt))

#Tian

eta=np.exp(dt*sigma**2);
u=0.5*np.exp(r*dt)*eta*(eta+1+np.sqrt(eta**2+2*eta-3));
d=0.5*np.exp(r*dt)*eta*(eta+1-np.sqrt(eta**2+2*eta-3));
p=(np.exp((b)*dt)-d)/(u-d);

#JR

u=np.exp(((b)-sigma**2/2)*dt+sigma*np.sqrt(dt));
d=np.exp(((b)-sigma**2/2)*dt-sigma*np.sqrt(dt));
p=(np.exp((b)*dt)-d)/(u-d);

for i in range(N+1):
    StockPrice=S*(u**i)*(d**(N-i))
    OptionValue[i]=np.max([F+Coupon,CRatio*StockPrice])
    if(OptionValue[i]==F+Coupon):
        ConversionProbability[i]=0
    else:
        ConversionProbability[i]=1


CouponTime=T2

for j in range(N, 0, -1):
    if(CouponTime != (T2 - np.ceil((N-j+1)*dt))):
        CouponPayment=1
        CouponTime=T2 - np.ceil((N-j)*dt)
        Time=CouponTime-dt*j
    else:
        CouponPayment=0
    for i in range(j):
        ru=ConversionProbability[i+1]*r+(1-ConversionProbability[i+1])*(r+k)
        rd=ConversionProbability[i]*r+(1-ConversionProbability[i])*(r+k)
        StockPrice=S*(u**i)*(d**(np.abs(i-j)))
        if (CouponPayment == 1):
            CouponValue=Coupon*(p*np.exp(-ru*Time)+(1-p)*np.exp(-rd*Time))
        else:
            CouponValue=0
        if (AmeEur=='e' or AmeEur=='a' and dt*j<=t1):
            OptionValue[i]=CouponValue+p*OptionValue[i+1]*np.exp(-ru*dt)+(1-p)*OptionValue[i]*np.exp(-rd*dt)
            ConversionProbability[i]=p*ConversionProbability[i+1]+(1-p)*ConversionProbability[i]
        elif(AmeEur=='a'):
            OptionValue[i]=np.max([CRatio*(StockPrice-X)+F,CouponValue+p*OptionValue[i+1]*np.exp(-ru*dt)+(1-p)*OptionValue[i]*np.exp(-rd*dt)])
            if(OptionValue[i]==CRatio*(StockPrice-X)+F):
                ConversionProbability[i]=1
            else:
                ConversionProbability[i]=p*ConversionProbability[i+1]+(1-p)*ConversionProbability[i]
print(OptionValue[0])

130.10946538112648


In [15]:
def print_haahtela_parameters(r, sigma, dt):
    b = r - q
    u = np.exp(r * dt) * (1 + np.sqrt(np.exp((sigma ** 2) * dt) - 1))
    d = np.exp(r * dt) * (1 - np.sqrt(np.exp((sigma ** 2) * dt) - 1))
    p = (np.exp((r - q) * dt) - d) / (u - d)

    # Stampa dei risultati
    print(f"=== Parametri del modello Haahtela ===")
    print(f"Up Factor (u): {u:.6f}")
    print(f"Down Factor (d): {d:.6f}")
    print(f"Probabilità (p): {p:.6f}")
print_haahtela_parameters(r, sigma, dt)


=== Parametri del modello Haahtela ===
Up Factor (u): 1.017413
Down Factor (d): 0.983587
Probabilità (p): 0.500000


In [ ]:
import numpy as np

def LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, model_type='', AmeEur='a', q=0, drift=0):
    """
    Lattice model for pricing a convertible bond with American or European options.

    Parameters:
    S0 : float : Initial stock price
    F : float : Face value of the bond
    T : float : Time to maturity in years
    r : float : Risk-free interest rate
    k : float : Credit spread or additional yield
    coupon : float : Annual coupon payment
    conversion_ratio : float : Number of shares per bond
    N : int : Number of time steps
    model_type : str : Type of binomial tree (e.g., 'CRR', 'EQP', 'Tian')
    AmeEur : str : 'a' for American, 'e' for European
    q : float : Dividend yield
    drift : float : Drift term for stock price evolution

    Returns:
    StockPrice : np.array : Stock price tree
    BondPrice : np.array : Convertible bond price tree
    ConversionProbability : np.array : Probability of conversion
    """
    
    dt = T / N  # Time step
    sigma = 0.2  # Assume a constant volatility for demonstration purposes

    # Calculate u, d, p using the specified model type
    u, d, p = calculate_tree_parameters(r, q, sigma, dt, model_type, drift, AmeEur='a')

    # Create the binomial stock price tree
    StockPrice = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            StockPrice[j, i] = S0 * (u ** j) * (d ** (i - j))

    # Create the bond price tree (including coupon payments)
    BondPrice = np.zeros((N + 1, N + 1))
    # Create the conversion probability tree
    ConversionProbability = np.zeros((N + 1, N + 1))

    # Calculate bond prices and conversion probabilities at maturity
    for j in range(N + 1):
        stock_price = StockPrice[j, N]
        bond_value_at_maturity = max(conversion_ratio * stock_price, F + coupon)
        BondPrice[j, N] = bond_value_at_maturity
        ConversionProbability[j, N] = 1 if conversion_ratio * stock_price > F + coupon else 0

    # Initialize coupon timing and time management
    CouponTime = T  # Initial coupon time matches maturity

    # Backward calculation for bond price and conversion probability
    for i in range(N - 1, -1, -1):
        if CouponTime != (T - np.ceil((N - i + 1) * dt)):
            CouponPayment = 1  # It's time to pay a coupon
            CouponTime = T - np.ceil((N - i) * dt)
            Time = CouponTime - dt * i
        else:
            CouponPayment = 0

        for j in range(i + 1):
            stock_price = StockPrice[j, i]

            # Dynamic discount factors (ru, rd)
            ru = ConversionProbability[j+1, i+1] * r + (1 - ConversionProbability[j+1, i+1]) * (r + k)
            rd = ConversionProbability[j, i+1] * r + (1 - ConversionProbability[j, i+1]) * (r + k)

            if CouponPayment == 1:
                CouponValue = coupon * (p * np.exp(-ru * Time) + (1 - p) * np.exp(-rd * Time))
            else:
                CouponValue = 0

            hold_bond_value = CouponValue + p * BondPrice[j + 1, i + 1] * np.exp(-ru * dt) + (1 - p) * BondPrice[j, i + 1] * np.exp(-rd * dt)
            convert_to_stock_value = conversion_ratio * stock_price

            # Modify for American or European option style
            if AmeEur == 'a':
                BondPrice[j, i] = max(hold_bond_value, convert_to_stock_value)
            else:
                BondPrice[j, i] = hold_bond_value

            # Update the conversion probability based on the exercise type
            if AmeEur == 'a' and BondPrice[j, i] == convert_to_stock_value:
                ConversionProbability[j, i] = 1
            else:
                ConversionProbability[j, i] = p * ConversionProbability[j + 1, i + 1] + (1 - p) * ConversionProbability[j, i + 1]

    return StockPrice, BondPrice, ConversionProbability


In [19]:
print(OptionValue[0])


130.10946538112648
